# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は機械翻訳にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は機械翻訳を行っていただきます。

機械翻訳は機械が言語を別の言語に翻訳するものです。

機械翻訳にはいくつか種類があるのでここでも紹介しておきます。

* PBMT(Phrase Base Machine Translation)モデル
 * [moses](http://www.statmt.org/moses/)というオープンソースで使用できるメジャーな機械翻訳のモデルですが、難しすぎて理解できない人を続出させる機械翻訳の鬼門です
* ニューラル機械翻訳
 * 翻訳元単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで翻訳先言語を学習させる手法

以下では、このChainerを利用しデータを準備するところから実際に言語モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#各種ライブラリ導入) 
2. [各値を設定](#各値を設定)
3. [モデルの学習](#モデルの学習)
4. [予測](#予測)
5. [実行](#実行)



## 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。



In [ ]:
import sys
import math
import numpy as np

from chainer import functions, optimizers

import util.generators as gens
from util.functions import trace, fill_batch
from util.model_file import ModelFile
from util.vocabulary import Vocabulary

from util.chainer_cpu_wrapper import wrapper

from EncoderDecoderModel import EncoderDecoderModel

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。


## 2.機械翻訳のクラス

下記を設定しています。
* ニューラルネットを用いて機械翻訳用のモデルを構成しています。
ややこしいので各構成の説明

全体構成

![](./pictures/chainer_machine_translation1.png)




## 3.各値を設定

各値を設定

* モードを学習かテストか設定
* 翻訳元言語の設定
* 翻訳先言語の設定
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定


In [ ]:
mode = "train"
source = "source_wakati_mecab.txt"
target = "target.txt"
vocab = 32768
embed = 256
hidden = 512
epoch = 100
minibatch = 64
generation_limit = 256

## 4.モデルの学習

学習用のメソッド

* 翻訳元言語を処理用の変数に変換
* 翻訳先言語を処理用の変数に変換
* 学習用のモデル設定

学習回数分、下記の処理を行う

* 翻訳元言語をlist化
* 翻訳先言語をlist化
* sorted_parellel処理はややこしいので少し解説
　翻訳元言語と翻訳先言語のリストを100×ミニバッチのサイズ分渡すとタプル形式でソートして返してくれます。
　それをbatch関数でミニバッチのサイズ分取得しているのがgen3の処理です。
 
* 初期化
* gen3を用いて翻訳元言語と翻訳先言語を取り出し
　すべての文字列の末尾に"</s>"を挿入する

* 仮説候補を取得
* 翻訳元言語、翻訳先言語、翻訳仮説を表示
* 各学習ごとにモデルを保存

In [ ]:
def train_model():
    trace('making vocaburaries ...')
    src_vocab = Vocabulary.new(gens.word_list(source), vocab)
    trg_vocab = Vocabulary.new(gens.word_list(target), vocab)

    trace('making model ...')
    model = EncoderDecoderModel.new(src_vocab, trg_vocab, embed, hidden)

    for i_epoch in range(epoch):
        trace('epoch %d/%d: ' % (i_epoch + 1, epoch))
        trained = 0
        gen1 = gens.word_list(source)
        gen2 = gens.word_list(target)
        gen3 = gens.batch(gens.sorted_parallel(gen1, gen2, 100 * minibatch), minibatch)
        model.init_optimizer()

        for src_batch, trg_batch in gen3:
            src_batch = fill_batch(src_batch)
            trg_batch = fill_batch(trg_batch)
            K = len(src_batch)
            hyp_batch = model.train(src_batch, trg_batch)

            for k in range(K):
                trace('epoch %3d/%3d, sample %8d' % (i_epoch + 1, epoch, trained + k + 1))
                trace('  src = ' + ' '.join([x if x != '</s>' else '*' for x in src_batch[k]]))
                trace('  trg = ' + ' '.join([x if x != '</s>' else '*' for x in trg_batch[k]]))
                trace('  hyp = ' + ' '.join([x if x != '</s>' else '*' for x in hyp_batch[k]]))

            trained += K

        trace('saving model ...')
        model.save(model + '.%03d' % (epoch + 1))

    trace('finished.')

## 5.予測

予測

* 学習したモデルを読み込む
* 翻訳元言語をミニバッチのサイズ分読み込んで、仮説候補をモデルから予測
* 仮説候補を表示


In [ ]:
def test_model(args):
    trace('loading model ...')
    model = EncoderDecoderModel.load(model)
    
    trace('generating translation ...')
    generated = 0

    with open(target, 'w') as fp:
        for src_batch in gens.batch(gens.word_list(source), minibatch):
            src_batch = fill_batch(src_batch)
            K = len(src_batch)

            trace('sample %8d - %8d ...' % (generated + 1, generated + K))
            hyp_batch = model.predict(src_batch, generation_limit)

            for hyp in hyp_batch:
                hyp.append('</s>')
                hyp = hyp[:hyp.index('</s>')]
                print(' '.join(hyp), file=fp)

            generated += K

    trace('finished.')

## 6.実行

In [ ]:
def main():

    trace('initializing ...')
    wrapper.init()

    if mode == 'train': train_model()
    elif mode == 'test': test_model()


if __name__ == '__main__':
    main()